In [1]:
from z3 import *
from itertools import combinations

Read instance file:

In [2]:
input_filename = '../../Instances/15x15.txt'

w, h, n, DX, DY = None, None, None, None, None

with open(input_filename, 'r') as f_in:
    lines = f_in.read().splitlines()

    split = lines[0].split(' ')
    w = int(split[0])
    h = int(split[1])

    n = int(lines[1])

    DX = []
    DY = []

    for i in range(int(n)):
        split = lines[i + 2].split(' ')
        DX.append(int(split[0]))
        DY.append(int(split[1]))

Solver:

In [3]:
solver = Solver()

Model:

In [4]:
XY = [(Int(f'XY_{i}_0'), Int(f'XY_{i}_1')) for i in range(n)]
XY

[(XY_0_0, XY_0_1),
 (XY_1_0, XY_1_1),
 (XY_2_0, XY_2_1),
 (XY_3_0, XY_3_1),
 (XY_4_0, XY_4_1),
 (XY_5_0, XY_5_1),
 (XY_6_0, XY_6_1),
 (XY_7_0, XY_7_1),
 (XY_8_0, XY_8_1),
 (XY_9_0, XY_9_1)]

In [5]:
# Non-overlapping constraint
for (i, j) in combinations(range(n), 2):
    solver.add(Or(XY[i][0] + DX[i] <= XY[j][0], 
                  XY[j][0] + DX[j] <= XY[i][0],
                  XY[i][1] + DY[i] <= XY[j][1],
                  XY[j][1] + DY[j] <= XY[i][1]))

In [6]:
# Boundaries consistency constraint
for i in range(n):
    solver.add(XY[i][0] >=0)
    solver.add(XY[i][1] >= 0)
    solver.add(XY[i][0] + DX[i] <= w)
    solver.add(XY[i][1] + DY[i] <= h)

In [7]:
%%time
solver.check()

CPU times: user 43.5 ms, sys: 400 µs, total: 43.9 ms
Wall time: 41.5 ms


sat

From Z3 model solution to file:

In [8]:
model = solver.model()

xy = [(model[XY[i][0]], model[XY[i][1]]) for i in range(n)]
xy

[(3, 0),
 (12, 0),
 (9, 7),
 (0, 0),
 (9, 0),
 (12, 4),
 (0, 6),
 (3, 3),
 (6, 0),
 (9, 12)]

In [9]:
output_filename = '../../pwp_utilities/15x15_sol.txt'
with open(output_filename, 'w') as f_out:
    f_out.write('{} {}\n'.format(w, h))
    f_out.write('{}\n'.format(n))
    for i in range(n):
        f_out.write('{} {}\t{} {}\n'.format(DX[i], DY[i], xy[i][0], xy[i][1]))